# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,20.73,70,5,4.38,PN,1697070799
1,1,hermanus,-34.4187,19.2345,15.63,84,98,3.11,ZA,1697070800
2,2,bethel,41.3712,-73.4140,15.17,79,0,2.06,US,1697070801
3,3,carrigaline,51.8117,-8.3986,9.86,93,75,3.09,IE,1697070802
4,4,ghadamis,30.1337,9.5007,21.66,40,0,4.14,LY,1697070802


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles= "OSM",
    frame_width = 800,
    frame_height = 600,
    size= "Humidity",
    color= "City"
)

# Display the map 
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values

no_clouds = city_data_df.loc[city_data_df["Cloudiness"] == 0, :]
ideal_temp = no_clouds.loc[no_clouds["Max Temp"].between(21, 27), :]
light_wind = ideal_temp.loc[ideal_temp["Wind Speed"] < 4.5, :]

# Drop any rows with null values
perfect_vaca = light_wind.dropna()

# Display sample data
perfect_vaca

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,ghadamis,30.1337,9.5007,21.66,40,0,4.14,LY,1697070802
7,7,tura,25.5198,90.2201,21.29,93,0,0.95,IN,1697070208
10,10,hassi messaoud,31.6804,6.0729,22.14,40,0,4.06,DZ,1697070806
24,24,chebba,35.2372,11.1150,22.48,68,0,1.54,TN,1697070815
113,113,ad dilam,23.9915,47.1627,26.23,16,0,2.38,SA,1697070874
114,114,macouria,4.9167,-52.3667,26.97,83,0,2.57,GF,1697070875
195,195,al kharijah,25.4514,30.5464,22.20,37,0,3.74,EG,1697070930
234,234,bushehr,28.9684,50.8385,23.05,83,0,0.00,IR,1697070957
278,278,la rinconada,37.4873,-5.9826,21.65,73,0,1.54,ES,1697070986
294,294,taroudant,30.4703,-8.8770,24.75,22,0,3.74,MA,1697070999


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_vaca[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,ghadamis,LY,30.1337,9.5007,40,
7,tura,IN,25.5198,90.2201,93,
10,hassi messaoud,DZ,31.6804,6.0729,40,
24,chebba,TN,35.2372,11.1150,68,
113,ad dilam,SA,23.9915,47.1627,16,
114,macouria,GF,4.9167,-52.3667,83,
195,al kharijah,EG,25.4514,30.5464,37,
234,bushehr,IR,28.9684,50.8385,83,
278,la rinconada,ES,37.4873,-5.9826,73,
294,taroudant,MA,30.4703,-8.8770,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row.Lng
    latitude = row.Lat
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filters"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ghadamis - nearest hotel: فندق عين الفارس
tura - nearest hotel: Eco Park Rest House
hassi messaoud - nearest hotel: AMC
chebba - nearest hotel: منتزه و نزل الكاهنة
ad dilam - nearest hotel: شقق فندقية
macouria - nearest hotel: La Chaumière
al kharijah - nearest hotel: Kharga Hotel
bushehr - nearest hotel: هتل رستوران سیراف
la rinconada - nearest hotel: Hotel Torre de los Guzmanes
taroudant - nearest hotel: Hotel Tiouit
saint-pierre - nearest hotel: Tropic Hotel
as sulayyil - nearest hotel: No hotel found
sao geraldo do araguaia - nearest hotel: Tomazi Hotel
mulayjah - nearest hotel: My Hotel
cambria - nearest hotel: Olallieberry Inn
nagar - nearest hotel: No hotel found
noosa heads - nearest hotel: Costa Nova Apartments
la cruz - nearest hotel: Hotel Las Palmas
crotone - nearest hotel: Hotel Concordia
ma'rib - nearest hotel: No hotel found
el hadjira - nearest hotel: El Hadjira
jijel - nearest hotel: Hôtel Louiza
maua - nearest hotel: Go Inn Santo André
jacobabad 

,City,Country,Lat,Lng,Humidity,Hotel Name
4,ghadamis,LY,30.1337,9.5007,40,فندق عين الفارس
7,tura,IN,25.5198,90.2201,93,Eco Park Rest House
10,hassi messaoud,DZ,31.6804,6.0729,40,AMC
24,chebba,TN,35.2372,11.1150,68,منتزه و نزل الكاهنة
113,ad dilam,SA,23.9915,47.1627,16,شقق فندقية
114,macouria,GF,4.9167,-52.3667,83,La Chaumière
195,al kharijah,EG,25.4514,30.5464,37,Kharga Hotel
234,bushehr,IR,28.9684,50.8385,83,هتل رستوران سیراف
278,la rinconada,ES,37.4873,-5.9826,73,Hotel Torre de los Guzmanes
294,taroudant,MA,30.4703,-8.8770,22,Hotel Tiouit


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
vaca_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles= "OSM",
    size= "Humidity",
    frame_width = 800,
    frame_height = 600,
    color= "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
vaca_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)